In [7]:
# /// script
# requires-python = ">=3.13"
# dependencies = [
#     "numpy",
#     "pyarrow",
# ]
# ///

In [8]:
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
from pathlib import Path

keys = [
    "GSM2219497_Cell_1",
    # "GSM2219498_Cell_2",
    # "GSM2219499_Cell_3",
    # "GSM2219500_Cell_4",
    # "GSM2219501_Cell_5",
    # "GSM2219502_Cell_6",
    # "GSM2219503_Cell_7",
    # "GSM2219504_Cell_8",
]

In [12]:
def separate_into_models(filepath):
    models_lines = {}
    with open(filepath, 'r') as file:    
        for line in file:
            line_annot = line[0:6].strip()
            if line_annot == "MODEL":
                model_num = line.split()[1]
                models_lines[model_num] = []
    
    with open(filepath, 'r') as file:
        current_model = "0"
        for line in file:
            line_annot = line[0:6].strip()
            if line_annot == "MODEL":
                model_num = line.split()[1]
                current_model = model_num
                continue
            if line_annot == "ENDMDL":
                continue
            if line_annot == "HETATM":
                models_lines[current_model].append(line)
    return models_lines

def model_lines_to_coord_arrays(lines):
    for line in lines:
        line_annot = line[0:6]
        # print(line)
        if line_annot == "HETATM" or line_annot == "ATOM":
            x = float(line[30:38])
            y = float(line[38:46])
            z = float(line[46:54])
            chrom = line[17:22]
            coord = int(line[80:90])
            # print(f'{chrom} {line[80:90]} -> {coord}')
            x_arr.append(x)
            y_arr.append(y)
            z_arr.append(z)
            chr_arr.append(chrom)
            coord_arr.append(coord)
    return [x_arr, y_arr, z_arr, chr_arr, coord_arr]

def arrays_to_table(x_arr, y_arr, z_arr, chrom_arr, coord_arr):
    x_nparr = np.array(x_arr)
    y_nparr = np.array(y_arr)
    z_nparr = np.array(z_arr)
    chrom_nparr = np.array(chrom_arr)
    coord_nparr = np.array(coord_arr)
    return pa.Table.from_arrays([x_nparr, y_nparr, z_nparr, chrom_nparr, coord_nparr], ["x", "y", "z", "chr", "coord"])

def write_to_file(outfile_path, table):
    with pa.OSFile(outfile_path, 'wb') as sink:
        with pa.ipc.RecordBatchFileWriter(sink, table.schema) as writer:
            writer.write_table(table)

basepath = "GSE80280_RAW"
postfix = "_genome_structure_model.pdb"
prefix = "Stevens-2017_"

out_folder = Path("out")
out_folder.mkdir(parents=True, exist_ok=True)

outputs_list_file = "out/out_file.txt"
with open(outputs_list_file, "w") as outputs_file:
    for key in keys:
        filepath = f"{basepath}/{key}{postfix}"
        print("input: " + filepath)
    
        lines = separate_into_models(filepath)
    
        for n, model in lines.items():
            x_arr = []
            y_arr = []
            z_arr = []
            chr_arr = []
            coord_arr = []
            [x_arr, y_arr, z_arr, chr_arr, coord_arr] = model_lines_to_coord_arrays(model)
            table = arrays_to_table(x_arr, y_arr, z_arr, chr_arr, coord_arr)
            
            outputpath_arrow = f'out/{prefix}{key}_model_{n}.arrow'
            print("Writing: " + outputpath_arrow)
            write_to_file(outputpath_arrow, table)
            
            outputs_file.write(f'{prefix}{key}_model_{n}\n')


input: GSE80280_RAW/GSM2219497_Cell_1_genome_structure_model.pdb
Writing: out/Stevens-2017_GSM2219497_Cell_1_model_1.arrow
Writing: out/Stevens-2017_GSM2219497_Cell_1_model_2.arrow
Writing: out/Stevens-2017_GSM2219497_Cell_1_model_3.arrow
Writing: out/Stevens-2017_GSM2219497_Cell_1_model_4.arrow
Writing: out/Stevens-2017_GSM2219497_Cell_1_model_5.arrow
Writing: out/Stevens-2017_GSM2219497_Cell_1_model_6.arrow
Writing: out/Stevens-2017_GSM2219497_Cell_1_model_7.arrow
Writing: out/Stevens-2017_GSM2219497_Cell_1_model_8.arrow
Writing: out/Stevens-2017_GSM2219497_Cell_1_model_9.arrow
Writing: out/Stevens-2017_GSM2219497_Cell_1_model_10.arrow
